# Importing Header and SoilPrep

In [1]:
import import_ipynb 
from Header import *

importing Jupyter notebook from Header.ipynb


In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


## Loading Data and MetaData

In [3]:
import import_ipynb
from LoadDataMetaData import *

importing Jupyter notebook from LoadDataMetaData.ipynb


In [4]:
import os
os.system('say "your Meta Tree started building"')
import time

In [5]:
# prepare_spec = ['none', 'fod2', 'continuum', 'log']
# prepare_target = ['none', 'minmax']

def find_X(p, n):
    if p == 'fod2':
        X = fod_sampled[n]
    elif p == 'continuum':
        X = sampled_cr[n]
    elif p == 'log':
        X = sampled_log[n]
    else:
        X = sampled_spec[n]
    return X

def find_spec(p, n, m):
    if n == 0:
        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
            spec = find_X(p, 100)
        else:
            spec = find_X(p,n)            
    else:
        spec = find_X(p,n)
    return spec    


def find_y(t):
    i = target_names.index(t) 
    y = T[i]
    return y
        
        

In [26]:
# 1. Available machine learning regression models --------------------------------- (1)
ml_methods = ['mult', 'svr',  'ridge']

# 2. Names of target variables in the dataframe ----------------------------------- (2)
target_names = ['Sand', 'Clay', 'TOC']

# 4. Available preprocessing for Spectral data ------------------------------------ (4)
prepare_spec = ['cr', 'cr2']

## Recording Hypertuning Parameters

In [7]:
from sklearn.metrics import make_scorer, r2_score

model ={}
param_grid ={}
model['mult'] = LinearRegression()
param_grid['mult'] = { 'fit_intercept': [True, False] }

model['ridge'] = KernelRidge()
param_grid['ridge']={'alpha': [ 0.00001,0.00002, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.002, \
                               0.005,  0.01, 0.05, 0.1, 0.5, 1]}

model['plsr'] = PLSRegression()
param_grid['plsr']={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


model['cubist'] = Cubist(random_state = 42)  
param_grid['cubist'] = { 'n_committees': [5, 10, 15, 20], 'n_rules': [10, 20, 30, 40, 50] }
                        

model['gbrt'] = GradientBoostingRegressor(random_state = 42)
param_grid['gbrt'] = {
    'n_estimators': [20,30,50],        # Number of boosting stages to be used
    'learning_rate': [0.001, 0.01, 0.1],     # Step size shrinkage used in updating weights
    'max_depth': [2, 3],                # Maximum depth of individual trees
    'min_samples_split': [2, 3],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2],         # Minimum number of samples required to be at a leaf node
}

model['svr'] = SVR()
param_grid['svr'] = {
    'C': [0.1, 1, 10],              # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Kernel function
    'gamma': [0.01, 0.1, 1],      # Kernel coefficient (only for 'rbf' kernel)
}



### Finding Hypertuned Parameters

In [8]:
# def find_core_model_param_on (m, t, p, tp, n):
#     Model = model[m]
#     X = find_spec(p,n,m)
#     y = find_y(t, tp)
    
#     row, col = X.shape

#     scorer = make_scorer(mean_squared_error, greater_is_better=False)
#     cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
#     grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
#                                 scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
#     grid_search.fit(X,y)
    
#     return (grid_search.best_params_)

def find_tuned_param_on (X_train, y_train, m):
    Model = model[m]
    X = X_train
    y = y_train
    
    row, col = X.shape

    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
    grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
                                scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
    grid_search.fit(X,y)
    
    return (grid_search.best_params_)
    

### Creating Hypertuned Models

In [9]:
def create_core_model_on (m, BP):
    if m == 'mult':
        Model = LinearRegression()
        
    elif m == 'ridge':
        Model =  KernelRidge(alpha = BP['alpha'])
        
    elif m == 'plsr':
        Model = PLSRegression(n_components = BP['n_components'])
        
    elif m == 'cubist':
        Model =  Cubist( n_committees = BP['n_committees'], n_rules = BP['n_rules'], random_state = 42) 
    
    elif m == 'gbrt':
        Model = GradientBoostingRegressor(n_estimators = BP['n_estimators'], learning_rate = BP['learning_rate'], \
                                          max_depth = BP['max_depth'], min_samples_split = BP['min_samples_split'], \
                                          min_samples_leaf = BP['min_samples_leaf'],  random_state = 42)
    elif m == 'svr':
        Model = SVR(C = BP['C'], kernel = BP['kernel'], gamma = BP['gamma'])
        
    return Model    
            

In [10]:
# def build_BPtree_for(method_name):
#     tree ={}
#     start = time.time()
#     m = method_name
#     #-- code to build tree----
#     for t in target_names:
#         print('BPtree for: '+ m +' ------> running on: ' + t)
#         tree[t] ={}
#         for tp in prepare_target:
#             tree[t][tp] ={}
#             for p in prepare_spec:
#                 tree[t][tp][p] ={}
#                 #---- find appropriate initial core model parameters-----------------
#                 if m == 'cubist' or m == 'gbrt':
#                     BP100 = find_core_model_param_on (m, t, p, tp, 100)
                
#                 for n in nbands_sampling: 
#                     tree[t][tp][p][n] ={}
#                     Y = tree[t][tp][p][n]
#                     if m == 'cubist' or m == 'gbrt':
#                         Y['BP'] = BP100
#                     else:    
#                         BP = find_core_model_param_on (m, t, p, tp, n)
#                         print('n_band:', n, '---> target:', t )
#                         Y['BP'] = BP
#     end = time.time() 
#     print('End time - Start time =', (end-start)) 
    
#     return (tree.copy())  

# def L1_y_pred (spectra, target, method, bp):
#     m = method
#     (r,c) = spectra.shape
    
#     Y_test = target.copy()
#     Y_pred = target.copy()
    
#     for i in range (0,r): 
#         #print('i: ', i)
#         full_spec = spectra.copy()
#         X_train = full_spec.drop(full_spec.index[i], axis=0)
#         X_test = full_spec.iloc[[i],:].copy()
#         full_tar = target.copy()
#         y_train = full_tar.drop(full_tar.index[i], axis=0)
#         y_test = full_tar.iloc[i].copy() 
        
#         #---- Model Creation, fitting, and predictions--------
#         Model = create_core_model_on (m, bp)
#         Model.fit(X_train, y_train)
#         y_pred = Model.predict(X_test)
#         Y_pred.iloc[i] = y_pred
        
#         return (Y_pred)      

In [11]:
#BPtree = {}

In [12]:
#BPtree['ridge'] = build_BPtree_for('ridge')

In [13]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [14]:
# BPtree['plsr'] = build_BPtree_for('plsr')

In [15]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [16]:
# BPtree['svr'] = build_BPtree_for('svr')

In [17]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

In [18]:
#BPtree['mult'] = build_BPtree_for('mult')

In [19]:
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)

## Building HMtree and BPtree

In [20]:
def build_HMtree_on (method_name, target):
    tree ={}
    bp_tree ={}
    
    start = time.time()
    m = method_name
    t = target
    
    print('tree for: '+ m +' ------> running on: ' + t)
    
    for tp in prepare_target:
        tree[tp] ={}
        bp_tree[tp] ={}
        for p in prepare_spec:
            tree[tp][p] ={}
            bp_tree[tp][p] ={}
            for n in nbands_sampling:
                tree[tp][p][n] ={}
                Y = tree[tp][p][n]
                
                bp_tree[tp][p][n] ={}
                    
                        
                #------ setting spec to appropriate (sampled) spectra----
                
                spec = find_spec(p, n, m)                
                                        
                #---- target selection and normalization ---
                y = find_y(t)
                
                
                #---- performing train-test split----------------------
                X_train, X_test, y_train, y_test = train_test_split(spec, y, test_size= 0.3, random_state=42)
                            
                #----- hypertuning parameter, model creation, fitting and prediction ----
                
                #L1_y_pred (spectra, target, method, bp)
                #bp = BPtree[m][t][tp][p][n]['BP']
                
                #X1_train, X1_test, y1_train, y1_test = train_test_split(X_train, y_train, test_size= 0.5, random_state=42)
                
#                 if m == 'cubist' or m == 'gbrt':
#                     bp = find_tuned_param_on (X1_train, y1_train, m)
                    
#                 else:
#                     bp = find_tuned_param_on (X_train, y_train, m)
                
                bp = find_tuned_param_on (X_train, y_train, m)
                bp_tree[tp][p][n]['BP'] = bp
                
                print('Method: '+ m + '---> Target: '+ t+ '----> SpecProc: ' +p+ ' ---> n_band: ', n)
                Model = create_core_model_on (m, bp)
                
                Model.fit(X_train, y_train)
                
                y_pred = Model.predict(X_test)
                yhat_pred = Model.predict(X_train)
                    
                #----- Model accuracy evaluation and plotting datas-----------
                            
                Y['test'] = y_test
                Y['testP'] = y_pred
                Y['train'] = y_train
                Y['trainP'] = yhat_pred
                    
                Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                Y['r2_test'] = find_r2(y_pred, y_test)
                Y['rpd_test'] = find_rpd(y_pred, y_test)                    
                Y['rmse_test'] = find_rmse(y_pred, y_test)
                
                Y['r2_train'] = find_r2(yhat_pred, y_train)
                                                                
                                                                         
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return tree.copy(), bp_tree.copy()

In [21]:
def build_HMtree_for (method_name):
    tree ={}
    bp_tree ={}
    
    start = time.time()
    m = method_name
    for t in target_names:
        #print('tree for: '+ m +' ------> running on: ' + t)
        tree[t], bp_tree[t] = build_HMtree_on (m, t)
        
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            
    print('End time - Start time =', (end-start))     
    return tree.copy(), bp_tree.copy()

In [22]:
HMtree = {}
BPtree = {}

In [23]:
# with open ('HMtree.pickle', 'rb') as file:
#     HMtree = pickle.load(file)
    
# with open ('BPtree.pickle', 'rb') as file:
#     BPtree = pickle.load(file)  

## Creating different branches of Mtree (for separate methods)

In [25]:
HMtree['mult'], BPtree['mult'] = build_HMtree_for('mult')
#HMtree['plsr'], BPtree['plsr'] = build_HMtree_for('plsr')
HMtree['svr'] , BPtree['svr']= build_HMtree_for('svr')
HMtree['ridge'], BPtree['ridge'] = build_HMtree_for('ridge')

tree for: mult ------> running on: Sand
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  5
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  7
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  9
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr ---> n_band:  10
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---

Method: mult---> Target: Sand----> SpecProc: cr2 ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Sand----> SpecProc: cr2 ---> n_band:  100
End time - Start time = 17.461348056793213
tree for: mult ------> running on: Clay
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr ---> n_band:  5
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr ---> n_band:  7
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay-

Method: mult---> Target: Clay----> SpecProc: cr2 ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr2 ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr2 ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: Clay----> SpecProc: cr2 ---> n_band:  100
End time - Start time = 14.457394361495972
tree for: mult ------> running on: TOC
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr ---> n_band:  0
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr ---> n_band:  2
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr ---> n_band:  3
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC--

Method: mult---> Target: TOC----> SpecProc: cr2 ---> n_band:  70
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr2 ---> n_band:  80
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr2 ---> n_band:  90
Fitting 60 folds for each of 2 candidates, totalling 120 fits
Method: mult---> Target: TOC----> SpecProc: cr2 ---> n_band:  100
End time - Start time = 14.80724024772644
End time - Start time = 46.774221420288086
tree for: svr ------> running on: Sand
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr ---> n_band:  0
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr ---> n_band:  2
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr ---> n_band:  3
Fitting 60 folds for each of 18 candidates, tota

Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  50
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  55
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  60
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  70
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  80
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  90
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Sand----> SpecProc: cr2 ---> n_band:  100
End time - Start time = 104.41859984397888
tree for: svr ------> running on: Clay
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---

Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  40
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  45
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  50
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  55
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  60
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  70
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  80
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: Clay----> SpecProc: cr2 ---> n_band:  90
Fitting 60 folds for each of 18 

Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  37
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  39
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  40
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  45
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  50
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  55
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  60
Fitting 60 folds for each of 18 candidates, totalling 1080 fits
Method: svr---> Target: TOC----> SpecProc: cr2 ---> n_band:  70
Fitting 60 folds for each of 18 candidat

Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  31
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  33
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  35
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  37
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  39
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  40
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  45
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Sand----> SpecProc: cr2 ---> n_band:  50
Fitting 60 folds for ea

Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  27
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  29
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  30
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  31
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  33
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  35
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  37
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: Clay----> SpecProc: cr2 ---> n_band:  39
Fitting 60 folds for ea

Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  23
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  25
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  27
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  29
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  30
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  31
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  33
Fitting 60 folds for each of 14 candidates, totalling 840 fits
Method: ridge---> Target: TOC----> SpecProc: cr2 ---> n_band:  35
Fitting 60 folds for each of 14

In [ ]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
    
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)    

In [ ]:
#best_score_on (HMtree, 'Clay', 'mult', 'r2')

In [ ]:
# HMtree['plsr'], BPtree['plsr'] = build_HMtree_for('plsr')
# HMtree['svr'] , BPtree['svr']= build_HMtree_for('svr')
# HMtree['ridge'], BPtree['ridge'] = build_HMtree_for('ridge')

In [ ]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
    
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)    

In [27]:
def best_score_on (HMtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    #best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            for n in nbands_sampling:
                Y = HMtree[m][t][tp][p][n]
                #r2_train = Y['r2_train']
                r2_test = Y['r2_test']
                    
                if scorer == 'iqrp':
                    cur_score = Y['iqrp_test']
                elif scorer == 'rpd':
                    cur_score = Y['rpd_test']
                else:
                    cur_score = Y['r2_test']  
                    
                if cur_score > best_score:
                    best_score = cur_score
                    best_tp = tp
                    best_n = n
                    best_p = p
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands:', best_n]                                 
    return (param_list)          

In [28]:
def best_score_for (HMtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_score_on (HMtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

In [29]:
#ml_methods = ['mult', 'ridge', 'plsr', 'svr', 'cubist']

In [30]:
#Plotting Model Accuracy (ipywidgets)
def plot_model_acc (method, target, spec_preprocessing, n_bands):
    
    m = method
    t = target
    
    i = target_names.index(target)
    
    
    p = spec_preprocessing
    n = n_bands
    tp = 'none'
    
    Y = HMtree[m][t][tp][p][n]
    
    y_test = Y['test']
    y_pred = Y['testP']
#     y_train = Y['train']
#     yhat_pred = Y['trainP']
    
    if m == 'plsr':
        y_pred = y_pred[:,0]
#       n_com = Y['n_comp']
        
        #yhat_pred = yhat_pred[:,0]
    
    
    iqrp_test = Y['iqrp_test']
    r2_test = Y['r2_test']
    rpd_test = Y['rpd_test']
    
#     iqrp_train = find_iqrp(yhat_pred, y_train)
#     r2_train = find_r2(yhat_pred, y_train)
#     rpd_train = find_rpd(yhat_pred, y_train)
    
    y_tp = pd.DataFrame({'actual':y_test.values, 'predic': y_pred})
    z = np.polyfit(y_test, y_pred, 1)
    
#     yhat_tp = pd.DataFrame({'actual':y_train.values, 'predic': yhat_pred})
#     zhat = np.polyfit(y_train, yhat_pred, 1)
    
    fig, axes = plt.subplots(1,2, figsize=(18,8))
    
    #with plt.style.context(('ggplot')): ---- PLOT of test-prediction --------------------------------------
    y_tp.plot.scatter(ax= axes[0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0].plot(y_test, np.polyval(z, y_test),  c='blue', linewidth=1)
    axes[0].plot(y_test, y_test, color='green', linewidth=1)
    axes[0].tick_params(axis='both', labelsize=10)
    axes[0].text(0.05, 0.95, target_names[i]+' (Test Data)', transform=axes[0].transAxes, fontsize = 20, color = clr[i])
    axes[0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_test,3)), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.95, 0.15, 'Method: '+method, transform=axes[0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    
#     if method == 'plsr':
#         axes[0].text(0.95, 0.05, 'n_component={:.2f}'.format(n_com), transform=axes[0].transAxes, 
#                     horizontalalignment='right', fontsize = 12)
    
     #------------------------ Ploting R2 Vs N_bands-----------------------------------
    
    #--- data for creating n_bands vs r2 scores plot -------------------------------
    
    pY = HMtree[m][t]['none'][p]
        
    X = nbands_sampling
    
    Yr2 = []
    
    for j in X:
        Yr2.append(pY[j]['r2_test'])         
   
    j = 0
    while j < len(Yr2): 
        if Yr2[j] <= 0:
            Yr2[j] = 0
        
        j = j + 1
 
    #------------------------------ STEM PLOT ---- for accuracy Vs n_bands ---------------------------------- 
    
    axes[1].stem(X,Yr2)
    axes[1].tick_params(axis='both', labelsize=10)
    axes[1].text(0.00, 1.01,  target_names[i], transform=axes[1].transAxes,fontsize = 20, color = clr[i])
    axes[1].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1].text(0.17, 1.01, '(none)', transform=axes[1].transAxes, horizontalalignment='left', fontsize = 16)
    axes[1].text(0.60, -0.1, 'n_bands', transform=axes[1].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
                rotation='vertical', transform=axes[1].transAxes, fontsize = 16)
        
   
    return    

In [31]:
best_score_for (HMtree, 'Sand', 'r2')

For:Sand-> ['r2', 0.61, 'Spec:', 'cr', 'bands:', 7] :mult
For:Sand-> ['r2', 0.59, 'Spec:', 'cr', 'bands:', 0] :svr
For:Sand-> ['r2', 0.51, 'Spec:', 'cr', 'bands:', 13] :ridge


In [32]:
#best_score_for (HMtree, 'Silt', 'r2')

In [33]:
best_score_for (HMtree, 'Clay', 'r2')

For:Clay-> ['r2', 0.36, 'Spec:', 'cr', 'bands:', 5] :mult
For:Clay-> ['r2', 0.3, 'Spec:', 'cr', 'bands:', 0] :svr
For:Clay-> ['r2', 0.36, 'Spec:', 'cr', 'bands:', 5] :ridge


In [34]:
best_score_for (HMtree, 'TOC', 'r2')

For:TOC-> ['r2', 0.72, 'Spec:', 'cr', 'bands:', 11] :mult
For:TOC-> ['r2', 0.6, 'Spec:', 'cr', 'bands:', 0] :svr
For:TOC-> ['r2', 0.21, 'Spec:', 'cr', 'bands:', 70] :ridge


## Plotting Model Accuracy (ipywidgets)

In [35]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

interactive(children=(Dropdown(description='method', options=('mult', 'svr', 'ridge'), value='mult'), Dropdown…

<function __main__.plot_model_acc(method, target, spec_preprocessing, n_bands)>